# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Martín Torrico
- Nombre de alumno 2: Alejandra Toro


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/MartinTorricoP/Laboratorios_MDS7202)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [2]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [3]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.


El ambiente de Blackjack adjunto consiste en una partida de Blackjack, juego donde el objetivo es que la suma de tus cartas se mayor a la del dealer sin excederse de los 21 puntos. La formulación en MDP corresponde a :

* Estados:
  - Suma actual de las cartas del jugador: Valores entre el 4 al 21.
  - Carta visible del dealer: Valores del 1 al 10 (donde 1 es un As).
  - As utilizable: Valor 0 o 1 (siendo 1 cuando el jugador puede utilizar un As).

* Acciones:
 - Quedarse con la suma actual: 0 (stick).
 - Pedir otra carta: 1 (hit).

* Recompensas:
 - Ganar: +1.
 - Perder: -1.
 - Empatar: 0.
 - Ganar con blackjack natural: +1,5 (solo si natural=True).



#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [4]:
import gymnasium as gym
import numpy as np

# Variables para almacenar resultados
rewards = []
n = 5000 #numero de repeticiones

# Simulación de 5000 episodios con acciones aleatorias
for _ in range(n):
    obs = env.reset()  # Reiniciar el ambiente
    done = False
    total = 0

    while not done:
        action = env.action_space.sample() # Seleccionar una acción aleatoria (0 o 1)
        obs, reward, done, _, _ = env.step(action) #Ejecutar la acción
        total += reward

    rewards.append(total)

# Calcular promedio y desviación estándar de las recompensas
average_reward = np.mean(rewards)
std_deviation = np.std(rewards)

print(f"Promedio de las recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de las recompensas: -0.407
Desviación estándar de las recompensas: 0.8922729403047029


El desempeño de la política aleatoria es subóptimo.

Esto es, ya que contamos con un promedio de recompensas negativos (-0.407), indicando más pérdidas que ganancias, sumado a una alta variabilidad con una desviación estándar de 0.8922, debido a la naturaleza estocástica del juego.

Con esta política, podemos tener un resultado base el cual podemos mejorar, mostrandonos que no es un juego donde una buena estrategia es similar a lanzar una moneda, sino que la estrategia de cuándo pedir o cuándo quedarse es importante para tener mejores resultados.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
from stable_baselines3 import DQN

# Crear el modelo DQN
model = DQN("MlpPolicy", env, verbose=1)

# Entrenar el modelo
model.learn(total_timesteps=5000)

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [6]:
# Evaluar la política aprendida por DQN
dqn_rewards = []

for _ in range(n):
    obs, _ = env.reset()
    done = False
    total = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)  # Política DQN
        obs, reward, done, _, _ = env.step(action)
        total += reward

    dqn_rewards.append(total)

# Calcular promedio y desviación estándar de la política DQN
average_dqn_reward = np.mean(dqn_rewards)
std_dqn_deviation = np.std(dqn_rewards)

# Comparar resultados
print("Comparación de resultados:")
print(f"Política Aleatoria \nPromedio: {average_reward}, Desviación: {std_deviation}")
print(f"Política DQN \nPromedio: {average_dqn_reward}, Desviación: {std_dqn_deviation}")


Comparación de resultados:
Política Aleatoria 
Promedio: -0.407, Desviación: 0.8922729403047029
Política DQN 
Promedio: -0.1144, Desviación: 0.9521095735260726


Podemos ver que el desempeño del agente mejora considerablemente en cuanto el promedio de sus recompensas, subiendo su promedio a -0.1144, tomando deciciones estratégicas e inteligentes y con ello ganando más veces que tomando deciciones al azar.

Sin embargo, podemos ver también que el desempeño del agente tiene más variabilidad en sus resultados con su política, lo cual nos indica que puede ser refinado para mejorar su toma de decisiones.

Con lo anterior, este escenario es mejor al aleatorio, pero sería recomendable seguir mejorando su consistencia en el desempeño.



#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [7]:
def elegir_accion(estado, model):
    action, _ = model.predict(estado, deterministic=True)
    return action

estado1 = [6, 7, False]
accion1 = elegir_accion(estado1, model)

estado2 = [19, 3, True]
accion2 = elegir_accion(estado2, model)

In [8]:
estado_1 = [6, 7, False]
accion_1 = elegir_accion(estado_1, model)

print(f"Acción para el estado 1 (6, 7, sin as): {'Pedir carta' if accion_1 == 1 else 'Quedarse'}")

Acción para el estado 1 (6, 7, sin as): Quedarse


In [9]:
estado_2 = [19, 3, True]
accion_2 = elegir_accion(estado_2, model)

print(f"Acción para el estado 2 (19, 3, con as): {'Pedir carta' if accion_2 == 1 else 'Quedarse'}")

Acción para el estado 2 (19, 3, con as): Quedarse


En este caso, solo el estado 2 tiene sentido, mientras que el estado 1 no tiene sentido, por lo que sería interesante poder optimizar el modelo.

Esto es porque la suma en el primer caso del agente es muy baja y sería normal pedir una sigiente carta (si no pedimos, prácticamente perdemos directamente), mientras que en el estado 2 la suma del jugador es muy alta y seguramente sobrepase el 21 (y actualmente, la carta que tiene el dealer es muy baja).




### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [11]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [12]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i % 5 == 0], fps=15) # editado para mejor rendimiento

El parámetro continuous=True transforma el problema en un desafío más complejo al permitir un control más detallado del lander, lo cual también requiere métodos de aprendizaje más sofisticados.

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

El ambiente de LunarLander consiste en un problema de control de trayectoria de un cohete, donde el objetivo es aterrizar en una plataforma sin salir del área designada ni estrellarse. La formulación en MDP corresponde a:

* Estados:
  - Coordenadas x e y del lander (posición relativa al área de aterrizaje).
  - Velocidades lineales en x e y.
  - Ángulo de orientación del lander.
  - Velocidad angular.
  - Contacto de las patas con el suelo (dos valores booleanos).

* Acciones:
  - 0: No realizar ninguna acción.
  - 1: Activar el motor lateral izquierdo.
  - 2: Activar el motor principal.
  - 3: Activar el motor lateral derecho.

* Recompensas:
- Proximidad al área de aterrizaje: Incrementos/penalizaciones por acercarse/alejarse.
- Velocidad y orientación:
  - Penalización por velocidades altas y ángulos inadecuados.
- Uso de motores:
  - Penalización por activaciones de motores -0,03 para laterales y -0,3 para el principal.
- Aterrizajes:
  - Recompensa de +10 por cada pata en contacto con el suelo.
  - Recompensa de +100 por un aterrizaje exitoso.
  - Penalización de -100 por estrellarse.
  - Un episodio se considera resuelto si el agente acumula al menos 200 puntos.

**Diferencia de acciones con Blackjack**:

A diferencia de Blackjack, donde las acciones son discretas y binarias (pedir o quedarse), en LunarLander las acciones pueden ser discretas (selección de motores) o **continuas** (intensidad de empuje), dependiendo de la configuración del ambiente.


#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [13]:
import gymnasium as gym
import numpy as np

# Variables para almacenar resultados
rewards = []
n = 10  # número de repeticiones

# Simulación de 10 episodios con acciones aleatorias
for _ in range(n):
    obs = env.reset()  # Reiniciar el ambiente
    done = False
    total = 0

    while not done:
        action = env.action_space.sample()  # Seleccionar una acción aleatoria
        obs, reward, done, _, _ = env.step(action)  # Ejecutar la acción
        total += reward

    rewards.append(total)

# Calcular promedio y desviación estándar de las recompensas
average_reward = np.mean(rewards)
std_deviation = np.std(rewards)

print(f"Promedio de las recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")


Promedio de las recompensas: -208.50902921506776
Desviación estándar de las recompensas: 143.06927422727438


El rendimiento de esta política aleatoria nuevamente es subóptima como es esperado.

Podemos ver que en promedio se estrella el lander 2 veces por aterrizaje, lo cual es un muy mal desempeño para lo que queremos y que tiene una desviación estándar muy alta de sus recompenzas, con un valor de 143.

Claramente hay espacio de mejora y lo desarrollaremos a continuación:



#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
from stable_baselines3 import PPO

# Crear el modelo PPO
model = PPO("MlpPolicy", env, verbose=1)

# Entrenar el modelo
model.learn(total_timesteps=10000)

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [30]:
# Evaluar la política aprendida por PPO
ppo_rewards = []

for _ in range(n):
    obs, _ = env.reset()
    done = False
    total = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)  # Política PPO
        obs, reward, done, _, _ = env.step(action)
        total += reward

    ppo_rewards.append(total)

# Calcular promedio y desviación estándar de la política PPO
average_ppo_reward = np.mean(ppo_rewards)
std_ppo_deviation = np.std(ppo_rewards)

# Comparar resultados
print("Comparación de resultados:")
print(f"Política Aleatoria \nPromedio: {average_reward}, Desviación: {std_deviation}")
print(f"Política PPO \nPromedio: {average_ppo_reward}, Desviación: {std_ppo_deviation}")

Comparación de resultados:
Política Aleatoria 
Promedio: -208.50902921506776, Desviación: 143.06927422727438
Política PPO 
Promedio: -131.04745755518053, Desviación: 82.37868052944239


El performance de la gente nuevamente mejora, pero no de manera tan sustancial como en el caso del blackjack. En este caso, podemos ver que en promedio se estrella 1 vez el lander en vez de 2, lo cual es bueno, pero no óptimo. Además, viendo que su desviación tiene un valor menor, sabemos que este resultado es más consistente en el tiempo. Con esto, tenemos un mejor modelo que el baseline, pero con espacio a mejora.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, seed = 123)
model.learn(total_timesteps = 100000)

In [40]:
# Evaluar la política aprendida por PPO
ppo_opt_rewards = []

for _ in range(n):
    obs, _ = env.reset()
    done = False
    total = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)  # Política PPO
        obs, reward, done, _, _ = env.step(action)
        total += reward

    ppo_opt_rewards.append(total)

# Calcular promedio y desviación estándar de la política PPO
average_ppo_opt_reward = np.mean(ppo_opt_rewards)
std_ppo_opt_deviation = np.std(ppo_opt_rewards)

# Comparar resultados
print("Comparación de resultados:")
print(f"Política Aleatoria \nPromedio: {average_reward}, Desviación: {std_deviation}")
print(f"Política PPO \nPromedio: {average_ppo_reward}, Desviación: {std_ppo_deviation}")
print(f"Política PPO Optimizado \nPromedio: {average_ppo_opt_reward}, Desviación: {std_ppo_opt_deviation}")

Comparación de resultados:
Política Aleatoria 
Promedio: -208.50902921506776, Desviación: 143.06927422727438
Política PPO 
Promedio: -131.04745755518053, Desviación: 82.37868052944239
Política PPO Optimizado 
Promedio: 166.15260113088635, Desviación: 83.67149751757806


Finalmente, añadiendo más timesteps al entrenamiento, el modelo optimizado toma un promedio positivo, lo cual es un resultado bastante bueno en base a nuestro sistema de recompensas. Notamos que la desviación se mantiene similar al modelo sin optimizar, por lo que tiene una robustez similar en sus resultados (es una buena noticia también, ya que no perdemos consistencia a cambio de mejores resultados promedio).

In [41]:
export_gif(model, 5)

In [ ]:
from IPython.display import Image
Image(open('agent_performance.gif','rb').read())

![Agent Performance](agent_performance.gif)

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [20]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [21]:
%pip install --upgrade --quiet PyPDF2

In [4]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00


In [5]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [6]:
import PyPDF2
from PyPDF2 import PdfReader

doc_paths = ['/content/doc1.pdf', '/content/doc2.pdf'] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [9]:
!pip install --upgrade --quiet  langchain-google-genai

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
import PyPDF2
import os

In [13]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def read_pdf(file_path):
    reader = PyPDF2.PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

all_texts = []
for doc_path in doc_paths:
    doc_text = read_pdf(doc_path)
    all_texts.append(doc_text)

# dividir en chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = []
for text in all_texts:
    chunks.extend(text_splitter.split_text(text))

print(f"Total chunks: {len(chunks)}")

docsearch = FAISS.from_texts(chunks, embeddings)
#vectorstore = FAISS.from_document(documents=chunks, embedding=embeddings)

Total chunks: 2


#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [14]:
# retriever
retriever = docsearch.as_retriever(
    search_type="similarity",  # Mide similitud entre embeddings
    search_kwargs={"k": 3}    # Recupera los 3 documentos mas relevantes
)

In [118]:
# formateamos los documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# recuperacion y formato
retriever_chain = retriever | format_docs

# test
question = "¿Qué información relevante hay sobre el documento?"
formatted_context = retriever_chain.invoke(question)

print("Contexto formateado:")
print(formatted_context)

Contexto formateado:
Training language models to follow instructions
with human feedback
Long OuyangJeff WuXu JiangDiogo AlmeidaCarroll L. Wainwright
Pamela MishkinChong Zhang Sandhini Agarwal Katarina Slama Alex Ray
John Schulman Jacob Hilton Fraser Kelton Luke Miller Maddie Simens
Amanda AskellyPeter Welinder Paul Christianoy
Jan LeikeRyan Lowe
OpenAI
Abstract
Making language models bigger does not inherently make them better at following
a user’s intent. For example, large language models can generate outputs that
are untruthful, toxic, or simply not helpful to the user. In other words, these
models are not aligned with their users. In this paper, we show an avenue for
aligning language models with user intent on a wide range of tasks by ﬁne-tuning
with human feedback. Starting with a set of labeler-written prompts and prompts
submitted through the OpenAI API, we collect a dataset of labeler demonstrations
of the desired model behavior, which we use to ﬁne-tune GPT-3 using 

In [115]:
from langchain_core.prompts import PromptTemplate

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en responder preguntas basándote en los documentos proporcionados.
Por favor, utiliza toda la información relevante para generar una respuesta completa.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante:
{context}

Pregunta:
{question}

Respuesta:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

In [116]:
import time
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)


In [119]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

response = rag_chain.invoke(question)
print(response)

El documento describe Toolformer, un modelo de lenguaje que aprende a usar herramientas externas a través de APIs simples.  Esto se logra de forma autosupervisada, necesitando solo unos pocos ejemplos para cada API.  Toolformer decide qué APIs llamar, cuándo llamarlas, qué argumentos pasar y cómo incorporar los resultados en la predicción de tokens futuros.  Se incorporan varias herramientas, incluyendo una calculadora, un sistema de preguntas y respuestas, un motor de búsqueda, un sistema de traducción y un calendario.  Las evaluaciones muestran que Toolformer mejora sustancialmente el rendimiento en cero disparos en diversas tareas, a menudo compitiendo con modelos mucho más grandes, sin sacrificar sus capacidades básicas de modelado del lenguaje.  El documento también analiza las limitaciones del método, como la incapacidad de usar herramientas en cadena o de forma interactiva, y la ineficiencia en el muestreo de datos para ciertas APIs.



#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [120]:
tuple_list = [("What is the main goal of the Toolformer model introduced in the document?",
               "To enable language models to use tools via APIs"),
              ("How is InstructGPT trained?", "Using reinforcement learning from human feedback (RLHF)"),
              ("How does Toolformer handle math tasks?", "By using a calculator tool")]

In [121]:
for idx in range(len(tuple_list)):
  print('Respuestas doc:')
  print(rag_chain.invoke(tuple_list[idx][0]))
  print('*'*200)
  print('\n')

Respuestas doc1:
The main goal of the Toolformer model is to enable language models to use external tools via simple APIs, achieving improved zero-shot performance across various downstream tasks without sacrificing core language modeling abilities.  This is accomplished in a self-supervised way, requiring minimal human annotation.  The model learns to decide which APIs to call, when to call them, what arguments to pass, and how to incorporate results into future token prediction.

********************************************************************************************************************************************************************************************************


Respuestas doc1:
InstructGPT is trained using a three-step process:

**Step 1: Supervised Fine-Tuning (SFT)**.  Labelers provide demonstrations of the desired model behavior on a distribution of prompts (mostly English prompts submitted to the OpenAI API and some labeler-written prompts).  A pre-trained GPT-3

*Respuesta:* La solución entrega respuestas esperadas, y da mas contexto e información que la respuesta esperada

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [23]:
# hiperparametros
chunk_sizes = [100, 500, 1000]
chunk_overlaps = [50, 100, 200]
ks = [1, 3, 5]
search_types = ["similarity", "mmr"]

In [28]:
total_chunks = len(all_texts)

results = []

for chunk_size in chunk_sizes:
    for chunk_overlap in chunk_overlaps:
        # Validación: `chunk_overlap` debe ser menor que `chunk_size`
        if chunk_overlap >= chunk_size:
            print(f"Saltando configuración inválida: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
            continue

        # Crear el divisor de texto
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        chunks = text_splitter.split_text(" ".join(all_texts))  # Dividir los textos

        for k in ks:
            # Validación: `k` no puede exceder el número de chunks disponibles
            if k > len(chunks):
                print(f"Saltando configuración inválida: k={k}, total_chunks={len(chunks)}")
                continue

            for search_type in search_types:
                print(f"Evaluando: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}, k={k}, search_type={search_type}")

                # Crear el sistema de recuperación
                local_docsearch = FAISS.from_texts(chunks, embeddings)
                retriever = local_docsearch.as_retriever(search_type=search_type, search_kwargs={"k": k})

                # Crear el RAG chain
                rag_chain =(
                            {
                                "context": retriever_chain,
                                "question": RunnablePassthrough(),
                            }
                            | rag_prompt
                            | llm
                            | StrOutputParser())
                # Evaluar tiempo y resultados
                start_time = time.time()
                response = rag_chain.invoke(question)
                elapsed_time = time.time() - start_time

                # Registrar los resultados
                results.append({
                    "chunk_size": chunk_size,
                    "chunk_overlap": chunk_overlap,
                    "k": k,
                    "search_type": search_type,
                    "elapsed_time": elapsed_time,
                    "response": response
                })

# Guardar los resultados
import pandas as pd
results_df = pd.DataFrame(results)
results_df.to_csv("sensitivity_analysis.csv", index=False)
print("Resultados guardados en sensitivity_analysis.csv.")


Evaluando: chunk_size=100, chunk_overlap=50, k=1, search_type=similarity
Evaluando: chunk_size=100, chunk_overlap=50, k=1, search_type=mmr
Saltando configuración inválida: k=3, total_chunks=1
Saltando configuración inválida: k=5, total_chunks=1
Saltando configuración inválida: chunk_size=100, chunk_overlap=200
Saltando configuración inválida: chunk_size=100, chunk_overlap=300
Evaluando: chunk_size=500, chunk_overlap=50, k=1, search_type=similarity
Evaluando: chunk_size=500, chunk_overlap=50, k=1, search_type=mmr
Saltando configuración inválida: k=3, total_chunks=1
Saltando configuración inválida: k=5, total_chunks=1
Evaluando: chunk_size=500, chunk_overlap=200, k=1, search_type=similarity
Evaluando: chunk_size=500, chunk_overlap=200, k=1, search_type=mmr
Saltando configuración inválida: k=3, total_chunks=1
Saltando configuración inválida: k=5, total_chunks=1
Evaluando: chunk_size=500, chunk_overlap=300, k=1, search_type=similarity
Evaluando: chunk_size=500, chunk_overlap=300, k=1, sear

In [30]:
display(results_df)

,chunk_size,chunk_overlap,k,search_type,elapsed_time,response
0,100,50,1,similarity,37.830560,"El documento describe Toolformer, un modelo de..."
1,100,50,1,mmr,87.197456,"El documento describe Toolformer, un modelo de..."
2,500,50,1,similarity,356.319088,"El documento describe Toolformer, un modelo de..."
3,500,50,1,mmr,128.108570,"El documento describe Toolformer, un modelo de..."
4,500,200,1,similarity,54.188279,"El documento describe Toolformer, un modelo de..."
5,500,200,1,mmr,195.108940,"El documento describe Toolformer, un modelo de..."
6,500,300,1,similarity,352.646452,"El documento describe Toolformer, un modelo de..."
7,500,300,1,mmr,48.887038,"El documento describe Toolformer, un modelo de..."
8,1000,50,1,similarity,64.943150,"El documento describe Toolformer, un modelo de..."
9,1000,50,1,mmr,27.520940,"El documento describe Toolformer, un modelo de..."


In [32]:
results_df['response'][0]

'El documento describe Toolformer, un modelo de lenguaje que aprende a usar herramientas externas a través de APIs simples.  A diferencia de enfoques previos que requieren grandes cantidades de anotaciones humanas o limitan el uso de herramientas a entornos específicos, Toolformer aprende de forma autosupervisada con solo unos pocos ejemplos para cada API.  Incorpora una variedad de herramientas, incluyendo una calculadora, un sistema de preguntas y respuestas, un motor de búsqueda, un sistema de traducción y un calendario.  Los experimentos demuestran que Toolformer, basado en un modelo GPT-J preentrenado con 6.7B parámetros, logra un rendimiento significativamente mejorado en varias tareas, a menudo competitivo con modelos mucho más grandes, sin sacrificar sus capacidades básicas de modelado del lenguaje.  El enfoque se basa en la capacidad de aprendizaje en contexto de los grandes modelos de lenguaje para generar conjuntos de datos completos desde cero.  Se muestra que el modelo dec

In [34]:
import plotly.express as px

fig = px.scatter(results_df, x="chunk_size", y="elapsed_time", color="search_type",
                 title="Tamaño del Contexto vs Tiempo de Respuesta",
                 labels={"chunk_size": "Tamaño del Contexto", "elapsed_time": "Tiempo (s)"})
fig.show()


*Respuesta:* Al analizar el grafico "Tamaño del Contexto vs Tiempo de Respuesta", se observa que el tamano de los chunks influye significativamente en el desempeno del RAG (Retrieval-Augmented Generation). Cuando los chunks son mas grandes, se reduce la cantidad de fragmentos generados, lo que puede agilizar el proceso de busqueda y reducir el tiempo de respuesta. Sin embargo, si los chunks son demasiado grandes, podrian incluir informacion irrelevante, disminuyendo la precision de las respuestas. Por otro lado, cuando los chunks son pequenos, se genera un mayor numero de fragmentos, lo que aumenta la granularidad de la busqueda pero puede incrementar el tiempo de respuesta debido a la necesidad de procesar mas datos.

Cuando se devuelven muchos chunks, el modelo puede recibir un contexto excesivamente largo, aumentando el tiempo de respuesta y posiblemente superando el limite de tokens manejado por el modelo. En cambio, si se devuelven pocos chunks, el contexto podria ser insuficiente para generar respuestas completas o precisas.

Finalmente, el tipo de busqueda (similarity vs mmr) tambien afecta los resultados. La busqueda por similarity tiende a devolver los documentos mas cercanos en terminos de embeddings, mientras que MMR (Maximal Marginal Relevance) prioriza la diversidad en los resultados. Esto ultimo puede ser util para evitar redundancias en las respuestas, pero podria aumentar el tiempo de procesamiento debido al calculo adicional necesario para garantizar la diversidad.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [122]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=1)


#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [123]:
!pip install wikipedia

In [124]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia_tool = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100, lang="es")
wikipedia_query_tool = WikipediaQueryRun(api_wrapper=wikipedia_tool)  # Use WikipediaQueryRun


#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [125]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import create_react_agent, AgentExecutor

In [126]:
prompt = PromptTemplate.from_template("""
Eres un agente experto en responder preguntas utilizando herramientas disponibles, que son las siguientes:
- Usa Tavily para preguntas relacionadas con eventos recientes, noticias, o información específica de internet.
- Usa Wikipedia para preguntas relacionadas con temas históricos, enciclopédicos o científicos.

Proporciona respuestas claras y completas en español.
Pregunta: {input}
{agent_scratchpad}
""")

In [127]:
tools = [tavily_tool, wikipedia_query_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [128]:
response = agent_executor.invoke({"input": "¿Cuáles son las noticias más recientes sobre el cambio climático?"})
print(response["output"])




> Entering new AgentExecutor chain...
Para responder a tu pregunta sobre las noticias más recientes sobre el cambio climático, utilizaré la herramienta Tavily, ya que se especializa en información de actualidad.  Sin embargo, la API `default_api` que tengo disponible no proporciona ejemplos de cómo usar `tavily_search_results_json`.  Necesito más información sobre el formato de la respuesta de `tavily_search_results_json` para poder procesarla y darte una respuesta completa.  Por favor, proporciona un ejemplo de la salida de `tavily_search_results_json` para una consulta de prueba, o indica cómo acceder a los datos relevantes dentro de la respuesta.


> Finished chain.
Para responder a tu pregunta sobre las noticias más recientes sobre el cambio climático, utilizaré la herramienta Tavily, ya que se especializa en información de actualidad.  Sin embargo, la API `default_api` que tengo disponible no proporciona ejemplos de cómo usar `tavily_search_results_json`.  Necesito más informaci

In [129]:
response = agent_executor.invoke({"input": "¿Qué es la inteligencia artificial?"})
print("Respuesta del agente:")
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Inteligencia artificial'}`
responded: Para responder a tu pregunta sobre qué es la inteligencia artificial, utilizaré Wikipedia, ya que se trata de un concepto científico y enciclopédico.



Page: Inteligencia artificial
Summary: La inteligencia artificial (IA), en el contexto de las cienciPara responder a tu pregunta sobre qué es la inteligencia artificial, utilizaré Wikipedia, ya que se trata de un concepto científico y enciclopédico.  La información que obtuve de Wikipedia indica que la inteligencia artificial (IA), en el contexto de las ciencias de la computación, es un campo de estudio que se enfoca en el desarrollo de sistemas informáticos capaces de realizar tareas que normalmente requieren inteligencia humana.  Esto incluye tareas como el aprendizaje, el razonamiento, la resolución de problemas y la percepción.  Sin embargo, la respuesta es incompleta debido a la limitación de la información proporc

In [130]:
response = agent_executor.invoke({"input": "¿Qué sabes sobre Tesla?"})
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tesla'}`
responded: Para responder a tu pregunta sobre Tesla, usaré ambas herramientas, ya que Tesla tiene aspectos históricos y también noticias recientes relevantes.

Primero, usaré Wikipedia para obtener información general sobre la historia y la empresa:



Luego, usaré Tavily para buscar información más actualizada sobre noticias o eventos recientes relacionados con Tesla:



Una vez que tenga la información de ambas fuentes, la combinaré para darte una respuesta completa y actualizada sobre Tesla.  Por favor, espera mientras proceso la información.




/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.




No good Wikipedia Search Result was found
Invoking: `tavily_search_results_json` with `{'query': 'Tesla'}`
responded: Para responder a tu pregunta sobre Tesla, usaré ambas herramientas, ya que Tesla tiene aspectos históricos y también noticias recientes relevantes.

Primero, usaré Wikipedia para obtener información general sobre la historia y la empresa:



Luego, usaré Tavily para buscar información más actualizada sobre noticias o eventos recientes relacionados con Tesla:



Una vez que tenga la información de ambas fuentes, la combinaré para darte una respuesta completa y actualizada sobre Tesla.  Por favor, espera mientras proceso la información.


[{'url': 'https://www.britannica.com/money/Tesla-Motors', 'content': 'Tesla, Inc. is an American manufacturer of electric vehicles, solar panels, and automobile batteries. It was founded in 2003 by American entrepreneurs Martin Eberhard and Marc Tarpenning and was named after Serbian American inventor Nikola Tesla. Elon Musk, an early in

*Respuesta:*

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [140]:
from langchain.tools import tool

@tool
def rag_tool(question: str) -> str:
    """
    Responde preguntas usando documentos vectorizados (solución RAG).
    """
    response = rag_chain.invoke(question)
    return response


In [141]:
@tool
def agent_tool(question: str) -> str:
    """
    Responde preguntas utilizando el agente con Tavily y Wikipedia.
    """
    response = agent_executor.invoke({"input": question})
    return response["output"]


#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [142]:
# Definir el prompt del enrutador
router_prompt = PromptTemplate.from_template("""
Eres un supervisor multiagente. Tu tarea es clasificar preguntas del usuario para decidir qué herramienta utilizar.
Usa las siguientes reglas:
- Usa "rag_tool" para preguntas relacionadas con los documentos cargados.
- Usa "agent_tool" para preguntas que requieren buscar en Tavily o Wikipedia.

Pregunta: {input}
{agent_scratchpad}

Responde con el nombre de la herramienta a utilizar:
""")

supervisor = create_tool_calling_agent(llm=llm, tools=[rag_tool, agent_tool], prompt=router_prompt)


supervisor_executor = AgentExecutor(agent=supervisor, tools=[rag_tool, agent_tool], verbose=True)

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [143]:
# test
# Pregunta que debería usar la solución RAG
response_rag = supervisor_executor.invoke({"input": "¿Qué información hay sobre los documentos cargados?"})
print("Respuesta RAG:", response_rag["output"])

# Pregunta que debería usar el agente (Tavily/Wikipedia)
response_agent = supervisor_executor.invoke({"input": "¿Qué es la inteligencia artificial?"})
print("Respuesta Agente:", response_agent["output"])



> Entering new AgentExecutor chain...

Invoking: `rag_tool` with `{'question': '¿Qué información hay sobre los documentos cargados?'}`


Based on the provided text, there is information about the following:

* **Dataset composition:** The dataset used for training and evaluation consists primarily of text prompts submitted to the OpenAI API, specifically those using earlier versions of the InstructGPT models on the Playground interface.  These prompts are diverse, including generation, question answering, dialogue, summarization, extraction, and other natural language tasks.  The dataset is over 96% English.  Labeler-written prompts were also used to bootstrap the process.  These were categorized as "Plain," "Few-shot," and "User-based."  The dataset was filtered for personally identifiable information (PII).  Table 1 shows the distribution of use-case categories for the API prompts, and Table 2 shows illustrative prompts.  More details on dataset sizes are in Table 6 and Appendix A.

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

El router se basa en reglas, definidas manualmente para decidir que tool o agente se debe manejar en cada tipo de pregunta, mientras que el multiagente con tools utiliza herramientas encapsuladas y un modelo LLM. El router es mas simple de implementar, pero el multiagente es mas flexible y escalable lo que permite agregar nuevas tools y adaptarse a preguntas ambiguas

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )